In [1]:
import pandas as pd
import numpy as np
import os
from markov_functions import *
#Change to Optimization
os.chdir("/home/adam/Documents/MarkovChainBaseball/Optimization//")

In [2]:
batting_data= import_raw_batting_data()

In [4]:
yankees_batting=find_team_atbats("NYA", batting_data)

In [6]:
yankees_players=make_team_transition_matrix("NYA")

In [16]:
yanks_starting9=['gardb001','ellsj001', 'sancg002',
                'beltc001','teixm001', 'casts001',  
                'gregd001', 'headc001', 'mccab002']
starting9_transitions=[yankees_players[x] for x in yanks_starting9]

In [51]:
def simulated_inning(transition_matrix_list, starting_batter):
    if len(transition_matrix_list)!=9:
        return "Can only play with 9 players!"
    possible_states=[(0,''), (0,'1'), (0,'2'), (0,'3'), (0, '12'), (0,'13'), (0,'23'), (0,'123'),
                        (1,''), (1,'1'), (1,'2'), (3,'3'), (1, '12'), (1,'13'), (1,'23'), (1,'123'),
                        (2,''), (2,'1'), (2,'2'), (2,'3'), (2, '12'), (2,'13'), (2,'23'),(2,'123'), 
                         '0', '1', '2', '3']
    
    current_state=np.random.choice(transition_matrix_list[starting_batter].index, size=1, 
                                   p=transition_matrix_list[starting_batter].iloc[0,:])[0]
    all_states=[(0,''),current_state]
    current_batter=starting_batter+1
    while current_state in possible_states[0:23]:
        next_state=np.random.choice(transition_matrix_list[current_batter].index, size=1, 
                                p=transition_matrix_list[current_batter].ix[[current_state], :].values[0])[0]
        all_states+=[next_state]
        current_state=next_state
        current_batter=(current_batter+1)%8
        runmatrix=run_matrix()
    runs=0
    for i in range(len(all_states)-1):
        runs+=runmatrix.ix[[all_states[i]], [all_states[i+1]]].values[0][0]
    return runs, all_states, current_batter

In [32]:
for i in range(30):
    print(simulated_inning_team(starting9_transitions, 0))

(0.0, [(0, ''), (0, '1'), (0, '12'), (1, '23'), (1, '123'), (2, '123')], 5)
(0.0, [(0, ''), (1, ''), (1, '1'), '0'], 3)
(0.0, [(0, ''), (1, ''), (1, '1'), '0'], 3)
(0.0, [(0, ''), (0, '1'), (1, '1'), '0'], 3)
(0.0, [(0, ''), (0, '1'), (2, ''), '0'], 3)
(2.0, [(0, ''), (0, '1'), (1, '1'), (2, ''), (2, ''), '0'], 5)
(4.0, [(0, ''), (1, ''), (1, '1'), (1, ''), (1, '1'), (1, ''), (1, '1'), (1, '12'), (2, '12'), '0'], 0)
(0.0, [(0, ''), (1, ''), (2, ''), '0'], 3)
(0.0, [(0, ''), (1, ''), (2, ''), '0'], 3)
(1.0, [(0, ''), (0, '1'), (1, '2'), (2, '2'), (2, '3'), '0'], 5)
(1.0, [(0, ''), (1, ''), (1, ''), (2, ''), '0'], 4)
(0.0, [(0, ''), (1, ''), (2, ''), '0'], 3)
(1.0, [(0, ''), (1, ''), (2, ''), (2, '2'), (2, '3'), '0'], 5)
(0.0, [(0, ''), (0, '1'), (1, '1'), (2, '1'), '0'], 4)
(2.0, [(0, ''), (0, ''), (0, '1'), (1, '1'), (1, '13'), (2, '13'), (2, '12'), '0'], 7)
(1.0, [(0, ''), (1, ''), (2, ''), (2, ''), '0'], 4)
(0.0, [(0, ''), (0, '1'), (1, '1'), (1, '12'), (2, '12'), '0'], 5)
(2.0, [(0,

In [54]:
def simulated_game(transition_matrix_list):
    """
    Given a list of transition matrices, this function will return a 
    """
    innings=[]
    inning=1
    batter=0
    while inning<=9:
        current=simulated_inning(transition_matrix_list, batter)
        innings+=[current]
        batter+=current[2]
        batter=batter%8
        inning+=1
    return sum([x[0] for x in innings])
        

In [64]:
simulated_game(starting9_transitions)

3.0